In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import pandas as pd
import cv2
import matplotlib.patches as patches
from PIL import Image
import matplotlib.pyplot as plt
import os
import numpy
from matplotlib import image
import warnings
# import openpyxl
import numpy as np
from PIL import ImageChops
import datetime
from matplotlib.animation import FuncAnimation, PillowWriter

Step1: Split video into frame, those frames will be feed into the network to detect gaze target.

In [7]:
# Input resource path, output path, and head_num (if there are 3 heads in the video, then use 3 times of this command with head_num 1,2,3) 
%run Data_Preprocessing.py --videoPath ./data/Raw_Video/videos/011.mp4  --imgPath ./output/image_frame/011/ --head_num 1


Now deal with video: 011
suc True
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 0
Save image into path: ./output/image_frame/011/0111000.jpg
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 1
Save image into path: ./output/image_frame/011/0111001.jpg
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 2
Save image into path: ./output/image_frame/011/0111002.jpg
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 3
Save image into path: ./output/image_frame/011/0111003.jpg
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 4
Save image into path: ./output/image_frame/011/0111004.jpg
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 5
Save image into path: ./output/image_frame/011/0111005.jpg
------------ ./output/image_frame/011/ ./data/Raw_Video/videos/011.mp4 1 6
Save image into path: ./output/image_frame/011/0111006.jpg
------------ ./output/image_

Step 2: Detect speaker in the video

In [14]:
%cd ./utils/Speaker_detector/
!pwd

/home/zzq/yuqi_summer_project/KETI_VGS/utils/Speaker_detector
/home/zzq/yuqi_summer_project/KETI_VGS/utils/Speaker_detector


In [19]:
!sh build_face_map_Generate_FaceMap.sh ../../data/Raw_Video/videos/011.mp4  011  ../../data/output/speaker_detect/

7417.29s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


==============================Now is working on run_pipeline_fixed.py=====================
2022-11-28 21:06:36.865688: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 21:06:36.952215: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-28 21:06:36.954995: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRAR

In [3]:
df = pd.read_csv('./output/KETI_Sequence.csv')
df_012 = df.loc[df['video_name'].str.contains('012')]
df_012

,Unnamed: 0,video_name,head_num,frame_seq,frame_name,head_position,gaze_point
5535,5535,012.mp4,1,0,0121000.jpg,"[409, 88, 591, 270]","[[413.36795, 177.376475]]"
5536,5536,012.mp4,1,1,0121001.jpg,"[407, 87, 593, 273]","[[413.61220000000003, 177.18496]]"
5537,5537,012.mp4,1,2,0121002.jpg,"[407, 88, 593, 274]","[[412.98134000000005, 177.30201499999998]]"
5538,5538,012.mp4,1,3,0121003.jpg,"[403, 86, 593, 276]","[[413.03425, 178.55066]]"
5539,5539,012.mp4,1,4,0121004.jpg,"[402, 89, 592, 279]","[[409.92175, 178.76619]]"
...,...,...,...,...,...,...,...
7080,7080,012.mp4,4,383,0124383.jpg,"[80, 150, 248, 318]","[[531.35175, 221.79672499999998]]"
7081,7081,012.mp4,4,384,0124384.jpg,"[80, 150, 248, 318]","[[531.733895, 222.19925]]"
7082,7082,012.mp4,4,385,0124385.jpg,"[81, 150, 249, 318]","[[531.557028, 220.8793875]]"
7083,7083,012.mp4,4,386,0124386.jpg,"[83, 153, 247, 317]","[[532.25965, 222.79869000000002]]"


In [4]:
# Get full name list concluding path of the images
def get_all_file(dir_name, file_extension):
    """
    Iterate Over Data
    """
    fullname_list, filename_list = [], []
    for root, dirs, files in os.walk(dir_name):
        for filename in files:
            if ("Detectors" not in os.path.join(root, filename)) and filename[-len(file_extension):] == file_extension:
                # fullname, concluding path
                fullname_list.append(os.path.join(root, filename))
                filename_list.append(filename)
    return fullname_list, filename_list

In [69]:
def bboxes_all(im, image_name, x_scale, y_scale, item, out_path):
    # Create figure and axes
    fig, ax = plt.subplots()
    # Display the image
    ax.imshow(im)
    width, height = im.size
    # Create a Rectangle patch
    rect = patches.Rectangle((item[0], item[1]), item[2]-item[0], item[3]-item[1], linewidth=1, edgecolor='g', facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    plt.plot(x_scale, y_scale, 'ro', color='g', markersize=8)
    plt.axis('off')
    #draw arrow
    norm_p = [x_scale, y_scale]
    plt.plot((norm_p[0],(item[0]+item[2])/2), (norm_p[1],(item[1]+item[3])/2), '-', color=(0,1,0,1))

    plt.savefig(out_path, bbox_inches='tight', dpi=fig.dpi, pad_inches=0.0)
    plt.clf()
    plt.cla()
    plt.close()
    # plt.imshow()
    return

In [6]:
path = './output/image_frame/012'

fullname_list, filename_list = get_all_file(path, 'jpg')

In [72]:
for frame_path,frame_name in zip(fullname_list, filename_list):
    # if frame_name == '0124001.jpg':
        frame_raw = Image.open(frame_path)
        frame_raw = frame_raw.convert('RGB')
        out_path = './output/demo_012/'+ frame_name
        width, height = frame_raw.size
        # print( width, height)

        # get gaze point
        try:
            gaze_box = df.loc[df['frame_name'].str.contains(frame_name[:-4])]['gaze_point'].tolist()
            new_list = [element.replace('[', '').replace(']', '') for element in gaze_box]
            x_scale = int(float(new_list[0].split(',')[0]))
            y_scale = int(float(new_list[0].split(',')[1]))

            #get head_bbox
            head_box = df.loc[df['frame_name'].str.contains(frame_name[:-4])]['head_position'].tolist()
            head_list = [element.replace('[', '').replace(']', '') for element in head_box]
            head_xmin = int(float(head_list[0].split(',')[0]))
            head_ymin = int(float(head_list[0].split(',')[1]))
            head_xmax = int(float(head_list[0].split(',')[2]))
            head_ymax = int(float(head_list[0].split(',')[3]))
            # head_box format[xmin,ymin,weight,height]
            head_bbox = [head_xmin,head_ymin,head_xmax,head_ymax]
        except IndexError:
            continue

        bboxes_all(frame_raw, frame_name, x_scale, y_scale, head_bbox, out_path)

1280 720


/tmp/ipykernel_1319014/2071158579.py:11: UserWarning: color is redundantly defined by the 'color' keyword argument and the fmt string "ro" (-> color='r'). The keyword argument will take precedence.
  plt.plot(x_scale, y_scale, 'ro', color='g', markersize=8)


1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1280 720
1

In [78]:
# generate gif
import imageio

gif_images = []
frame_num = 387
for i in range(0, frame_num):
    try:
        gif_images.append(imageio.imread('./output/demo_012/0124'+str(i).zfill(3)+'.jpg'))   
    except FileNotFoundError:
        continue
imageio.mimsave("./output/demo_012/0124.gif", gif_images, fps=20) 


/tmp/ipykernel_1319014/1141426445.py:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning dissapear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  gif_images.append(imageio.imread('./output/demo_012/0124'+str(i).zfill(3)+'.jpg'))
